# Parsing data

Загрузка ***[по ссылке](http://requests-world-bank.task-sss.krasilnikov.spb.ru/7cb3c7c270c0a64cb5d19dbe79e769c9)***, с помощью библиотеки requests файлов "__Best countries 2019__" и "__Best countries 2020__".

Все столбцы необходимо привести к соответствующим типам:
- Country - object
- все остальные столбцы - int

И отсортировать датафреймы по общему рейтингу (__Overall Rank__)

In [ ]:
import requests
import pandas as pd
import io

def create_df(url):
    csv_f = requests.get(url)
    bytes = io.BytesIO()
    bytes.write(csv_f.content)
    bytes.seek(0)
    df = pd.read_csv(bytes, sep=';')
    df[['Overall Rank', 'Entrepreneurship', 'Adventure', 'Citizenship', 'Cultural Influence', 'Heritage', 'Movers', 'Open for Business', 'Power', 'Quality of Life']] = df[['Overall Rank', 'Entrepreneurship', 'Adventure', 'Citizenship', 'Cultural Influence', 'Heritage', 'Movers', 'Open for Business', 'Power', 'Quality of Life']].astype(int)
    df.sort_values("Overall Rank")
    return df

best_2019 = create_df('http://requests-world-bank.task-sss.krasilnikov.spb.ru/static/2019_s.csv?key=best_countries_2019')
best_2020 = create_df('http://requests-world-bank.task-sss.krasilnikov.spb.ru/static/2020_s.csv?key=best_countries_2020')



**Задание 2** (**2 балла**)

Создание переменной __fb2020H__  со списком из 5 лучших стран по показателю __Heritage__ 2020 года.

Создание переменной __fw2019QL__ со списком из 5 худших стран по показателю __Quality of Life__ 2019 года.

In [ ]:
fb2020H = best_2020.sort_values('Heritage').head(5)['Country'].to_list()
fw2019QL = best_2019.sort_values('Quality of Life').tail(5)['Country'].to_list()

Загрузка файла "__World Development Indicators__" для 10 стран. Создание датафрейма __ten_countries__ с объединенной информацией из полученных файлов

In [ ]:
ten_counries_array = fb2020H + fw2019QL
base_url = 'http://requests-world-bank.task-sss.krasilnikov.spb.ru/givememycsv/7cb3c7c270c0a64cb5d19dbe79e769c9/'
def create_base_df():
    url = base_url + ten_counries_array[0]
    csv_f = requests.get(url)
    bytes = io.BytesIO()
    bytes.write(csv_f.content)
    bytes.seek(0)
    df = pd.read_csv(bytes, sep=';')
    df.pop('Unnamed: 0')
    abbr = list(df.columns.values)[1]
    df.rename(columns={abbr: ten_counries_array[0]}, inplace=True)
    return  df

ten_countries = create_base_df()

for i in ten_counries_array[1:]:
    url = base_url + i
    csv_f = requests.get(url)
    bytes = io.BytesIO()
    bytes.write(csv_f.content)
    bytes.seek(0)
    df = pd.read_csv(bytes, sep=';')
    abbr = list(df.columns.values)[2]
    ten_countries[i] = df[abbr]
ten_countries


,Series,Italy,Spain,Greece,France,Mexico,Pakistan,Nigeria,Angola,Jordan,Lebanon
0,AG.AGR.TRAC.NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AG.CON.FERT.PT.ZS,230.747598,114.592826,83.533155,386.227369,5.042637e+02,131.631587,91.410126,NaN,1.341468,34.363782
2,AG.CON.FERT.ZS,130.590510,157.698525,133.261780,172.679903,1.029283e+02,155.992657,19.737331,7.930094,114.044776,286.364848
3,AG.LND.AGRI.K2,124050.000000,261833.239000,61036.000000,286601.000000,1.068910e+06,363000.000000,691234.500000,569524.900000,10218.000000,6580.000000
4,AG.LND.AGRI.ZS,41.665267,52.408210,47.351435,52.341765,5.498650e+01,47.089041,75.895616,45.682594,11.509349,64.320626
...,...,...,...,...,...,...,...,...,...,...,...
1430,per_si_allsi.cov_q1_tot,NaN,NaN,NaN,NaN,4.619328e+00,NaN,NaN,NaN,NaN,NaN
1431,per_si_allsi.cov_q2_tot,NaN,NaN,NaN,NaN,9.351709e+00,NaN,NaN,NaN,NaN,NaN
1432,per_si_allsi.cov_q3_tot,NaN,NaN,NaN,NaN,1.414478e+01,NaN,NaN,NaN,NaN,NaN
1433,per_si_allsi.cov_q4_tot,NaN,NaN,NaN,NaN,1.749071e+01,NaN,NaN,NaN,NaN,NaN


Создание копии датафрейма __ten_countries__ под именем __ten_countries_full__ (датафрейм __ten_countries__ должен остаться неизменным). Поиск в датафрейме __ten_countries_full__ найдите индикатор c __индексом 5__ и подсчет для всех стран среднее, максимальное и минимальное значения этого индикатора


In [ ]:
ten_countries_full = ten_countries.copy()
ten_countries_full = ten_countries_full.dropna()
row_list = ten_countries_full.loc[5].to_list()
nums_list = []
for value in row_list:
    try:
        nums_list.append(float(value))
    except:
        continue
avg_ind = sum(nums_list)/len(nums_list)
min_ind = min(nums_list)
max_ind = max(nums_list)

Сохранение данных в формате Excel

In [ ]:
with pd.ExcelWriter('ten_countries_full.xlsx') as writer:
    ten_countries_full.to_excel(writer)
